In [1]:
workspace()

using BenchmarkTools

immutable SliceIterator{A<:AbstractArray}
  a::A
  length
  l
  r
end

In [14]:
Profile.clear()
function SliceIterator(a, dim::Int)
    #cols = ColVec(length(size(a)))
    #cols = Tuple{Int64,Colon}
    n = length(size(a))
    l = tuple(fill(:, dim-1)...)
    r = tuple(fill(:, n-dim)...)
    
    len = size(a,dim)
    
    SliceIterator(a, len, l,r)
end

Base.length(itr::SliceIterator) = itr.length
Base.size(itr::SliceIterator) = (length(itr),)
Base.start(itr::SliceIterator) = 1
Base.done(itr::SliceIterator, i::Int) = i > length(itr)
function Base.next(itr::SliceIterator, i::Int)
   slice(itr.a, itr.l..., i, itr.r...), i+1
end



A=rand(100000,1000)
mycollect(a) = [a[i,:] for i in 1:size(a,1)]
dims(i) = (i,:)
myslice(a) = [slice(a,dims(i)...) for i in 1:size(a,1)]

myslice (generic function with 1 method)

In [17]:
#@show @benchmark collect(A)
@show @benchmark (mycollect(A))
@show @benchmark collect(SliceIterator(A, 1))
@show @benchmark collect(myslice(A))

@benchmark(mycollect(A)) = BenchmarkTools.Trial: 
  samples:          3
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  778.95 mb
  allocs estimate:  498980
  minimum time:     1.95 s (33.18% GC)
  median time:      2.16 s (30.01% GC)
  mean time:        2.16 s (28.31% GC)
  maximum time:     2.37 s (29.79% GC)
@benchmark(collect(SliceIterator(A,1))) = BenchmarkTools.Trial: 
  samples:          103
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  16.01 mb
  allocs estimate:  499501
  minimum time:     26.26 ms (0.00% GC)
  median time:      50.62 ms (49.38% GC)
  mean time:        49.07 ms (42.91% GC)
  maximum time:     77.42 ms (55.83% GC)
@benchmark(collect(myslice(A))) = BenchmarkTools.Trial: 
  samples:          178
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  12.21 mb
  allocs estimate:  300004
  minimum time:     12.86 ms (0.00% GC)
  median

BenchmarkTools.Trial: 
  samples:          178
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  12.21 mb
  allocs estimate:  300004
  minimum time:     12.86 ms (0.00% GC)
  median time:      15.67 ms (0.00% GC)
  mean time:        26.35 ms (41.30% GC)
  maximum time:     55.91 ms (48.89% GC)

In [16]:
collect(SliceIterator(A, 1))

100000-element Array{Any,1}:
 [0.508649,0.511054,0.466657,0.882183,0.358672,0.655844,0.0174267,0.876219,0.472057,0.356085  …  0.525542,0.163685,0.866486,0.153948,0.205162,0.437046,0.0859482,0.908424,0.57249,0.613806]
 [0.508649,0.511054,0.466657,0.882183,0.358672,0.655844,0.0174267,0.876219,0.472057,0.356085  …  0.525542,0.163685,0.866486,0.153948,0.205162,0.437046,0.0859482,0.908424,0.57249,0.613806]
 [0.508649,0.511054,0.466657,0.882183,0.358672,0.655844,0.0174267,0.876219,0.472057,0.356085  …  0.525542,0.163685,0.866486,0.153948,0.205162,0.437046,0.0859482,0.908424,0.57249,0.613806]
 [0.508649,0.511054,0.466657,0.882183,0.358672,0.655844,0.0174267,0.876219,0.472057,0.356085  …  0.525542,0.163685,0.866486,0.153948,0.205162,0.437046,0.0859482,0.908424,0.57249,0.613806]
 [0.508649,0.511054,0.466657,0.882183,0.358672,0.655844,0.0174267,0.876219,0.472057,0.356085  …  0.525542,0.163685,0.866486,0.153948,0.205162,0.437046,0.0859482,0.908424,0.57249,0.613806]
 [0.508649,0.511054,0.466657,0.

In [14]:
Profile.clear()
A=rand(10000,100000)
x= @profile collect(SliceIterator(A,1))
typeof(x) |> dump
Profile.print()

Array
10 task.jl; anonymous; line: 447
 10 ...4/IJulia/src/IJulia.jl; eventloop; line: 143
  10 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 183
   10 loading.jl; include_string; line: 282
    10 profile.jl; anonymous; line: 16
     8 array.jl; collect; line: 255
     2 array.jl; collect; line: 256


In [15]:
x

10000-element Array{Any,1}:
 [0.394914,0.0602902,0.367433,0.56197,0.872677,0.591798,0.986725,0.956,0.104128,0.718466  …  0.157523,0.0279633,0.149594,0.180372,0.786194,0.0956629,0.22173,0.79015,0.580078,0.43116]         
 [0.266273,0.153137,0.297041,0.252344,0.924375,0.291009,0.383612,0.253188,0.626669,0.0533846  …  0.569131,0.179185,0.356106,0.178546,0.685984,0.719859,0.901177,0.908585,0.297581,0.245418]    
 [0.825087,0.743929,0.591186,0.370524,0.869099,0.379512,0.683651,0.521067,0.338911,0.87319  …  0.475297,0.275887,0.614288,0.948668,0.923538,0.158577,0.016667,0.118067,0.871264,0.533519]      
 [0.579695,0.345911,0.0305057,0.505408,0.707105,0.399805,0.397415,0.636237,0.0297708,0.593969  …  0.907708,0.262315,0.762959,0.76338,0.697364,0.20066,0.686961,0.476173,0.39956,0.0985477]     
 [0.223378,0.61412,0.920269,0.374923,0.669169,0.0925388,0.157307,0.615097,0.883653,0.578954  …  0.803862,0.122315,0.608658,0.484485,0.00777193,0.410008,0.667442,0.351693,0.833999,0.303883]   
 [0.45658,0.

In [5]:
dump(SliceIterator(A,1))

SliceIterator{Array{Float64,2}} 
  a: Array(Float64,(100,100)) 100x100 Array{Float64,2}:
 0.534806   0.617559   0.259453   …  0.72872    0.409102   0.750931 
 0.215744   0.809954   0.867613      0.909051   0.470718   0.446382 
 0.578762   0.146268   0.0580017     0.404426   0.0872413  0.850044 
 0.40591    0.694705   0.7085        0.39022    0.510348   0.10983  
 0.80562    0.950035   0.0782669     0.0701162  0.833876   0.622836 
 0.621973   0.740362   0.732713   …  0.352081   0.582872   0.858684 
 0.0160471  0.342949   0.550382      0.343565   0.465164   0.39128  
 0.454893   0.0452133  0.498127      0.932638   0.687688   0.659169 
 0.21681    0.189988   0.315131      0.498312   0.675978   0.460695 
 0.745072   0.347901   0.956502      0.508259   0.275172   0.350436 
 0.724201   0.919885   0.865974   …  0.600394   0.726582   0.715674 
 0.674326   0.227598   0.991319      0.848721   0.131149   0.0718255
 0.235139   0.955219   0.880506      0.348223   0.127967   0.061391 
 ⋮            

In [95]:
l=(:,:)
i=1
@show @benchmark tuple(l...,i)
t=tuple(l...,i)
t, typeof(t)

@benchmark(tuple(l...,i)) = BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     126
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  48.00 bytes
  allocs estimate:  2
  minimum time:     753.00 ns (0.00% GC)
  median time:      867.00 ns (0.00% GC)
  mean time:        881.76 ns (1.62% GC)
  maximum time:     72.41 μs (98.54% GC)


(Colon(),Colon(),1)

In [99]:
@benchmark l=(:,:)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     997
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  32.00 bytes
  allocs estimate:  1
  minimum time:     18.00 ns (0.00% GC)
  median time:      21.00 ns (0.00% GC)
  mean time:        31.53 ns (30.78% GC)
  maximum time:     7.56 μs (99.51% GC)

In [104]:
NTuple{2,Colon}((:,:))

(Colon(),Colon())

In [109]:
fill(:,2) |> tuple
@benchmark tuple(fill(:,2)...)

LoadError: LoadError: UndefVarError: @benchmark not defined
while loading In[109], in expression starting on line 2

In [13]:
SubArray{Float64, 1, typeof(A), (Int, Colon), 0}
#(A, (1,Colon()), NTuple((1,)), 1, 1)

LoadError: LoadError: TypeError: SubArray: in parameter, expected Type{T}, got Tuple{DataType,DataType}
while loading In[13], in expression starting on line 1

In [15]:
?SubArray

search: SubArray



No documentation found.

**Summary:**

```julia
immutable SubArray{T,N,P<:AbstractArray{T,N},I<:Tuple{Vararg{Union{AbstractArray{T,1},Colon,Int64}}},LD} <: AbstractArray{T,N}
```

**Fields:**

```julia
parent      :: P<:AbstractArray{T,N}
indexes     :: I<:Tuple{Vararg{Union{AbstractArray{T,1},Colon,Int64}}}
dims        :: NTuple{N,Int64}
first_index :: Int64
stride1     :: Int64
```


In [17]:
SubArray(A, (1,Colon()), NTuple((1,)), 1, 1)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{SubArray{T,N,P<:AbstractArray{T,N},I<:Tuple{Vararg{Union{AbstractArray{T,1},Colon,Int64}}},LD}}, ::Array{Float64,2}, ::Tuple{Int64,Colon}, ::Tuple{Int64}, ::Int64, ::Int64)
This may have arisen from a call to the constructor SubArray{T,N,P<:AbstractArray{T,N},I<:Tuple{Vararg{Union{AbstractArray{T,1},Colon,Int64}}},LD}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[17], in expression starting on line 1

In [39]:
A=reshape(1:16,4,4)
s = slice(A,2:3,1:4)
@show dump(s)
s

SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},1} 
  parent: Array(Int64,(4,4)) 4x4 Array{Int64,2}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16
  indexes: Tuple{UnitRange{Int64},UnitRange{Int64}} 
  dims: Tuple{Int64,Int64} 
  first_index: Int64 2
  stride1: Int64 1
dump(s) = nothing


2x4 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},1}:
 2  6  10  14
 3  7  11  15

In [43]:
A=reshape(1:16,4,4)
s = slice(A,:,2)
@show dump(s)
s

SubArray{Int64,1,Array{Int64,2},Tuple{Colon,Int64},2} 
  parent: Array(Int64,(4,4)) 4x4 Array{Int64,2}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16
  indexes: Tuple{Colon,Int64} 
  dims: Tuple{Int64} 
  first_index: Int64 5
  stride1: Int64 1
dump(s) = nothing


4-element SubArray{Int64,1,Array{Int64,2},Tuple{Colon,Int64},2}:
 5
 6
 7
 8

In [48]:
s.indexes[2]
s

4-element SubArray{Int64,1,Array{Int64,2},Tuple{Colon,Int64},2}:
 5
 6
 7
 8